
参考资料：

https://blog.csdn.net/Wiking__acm/article/details/48448933?utm_source=blogxgwz3

In [9]:
import pandas as pd

train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'])

In [35]:
Id = test.Id

In [10]:
weekdays = pd.get_dummies(train.DayOfWeek)
district = pd.get_dummies(train.PdDistrict)

In [11]:
weekdays.shape

(878049, 7)

In [12]:
district.shape

(878049, 10)

In [13]:
# 目标变量
y = train.Category

In [15]:
# 一共 39 个类别
len(y.unique())

39

In [16]:
X = pd.concat(
    [
        train.Dates.dt.year, train.Dates.dt.month, train.Dates.dt.day,
        train.Dates.dt.hour, weekdays, district
    ],
    axis=1)

In [17]:
pd.options.display.max_columns = 100

In [19]:
X.shape

(878049, 21)

In [20]:
X.head()

,Dates,Dates,Dates,Dates,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,2015,5,13,23,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,2015,5,13,23,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,2015,5,13,23,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,2015,5,13,23,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,2015,5,13,23,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


下面类似处理测试集。

In [21]:
weekdays = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)

data_test = pd.concat(
    [
        test.Dates.dt.year, test.Dates.dt.month, test.Dates.dt.day,
        test.Dates.dt.hour, weekdays, district
    ],
    axis=1)

In [22]:
data_test.shape

(884262, 21)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
y_pred = mnb.predict(X_test)

In [27]:
from sklearn.metrics import log_loss

predict_proba = mnb.predict_proba(X_test)
print(log_loss(y_test, predict_proba))

2.651474878145174


In [28]:
%%time
mnb = MultinomialNB()
mnb.fit(X, y)

CPU times: user 5.39 s, sys: 442 ms, total: 5.83 s
Wall time: 5.45 s


In [29]:
%%time
y_pred_proba = mnb.predict_proba(data_test)

CPU times: user 1.91 s, sys: 879 ms, total: 2.79 s
Wall time: 2.16 s


In [38]:
len(y_pred_proba)

884262

In [30]:
result = pd.DataFrame(y_pred_proba, columns=mnb.classes_)

In [32]:
# 输出要一分多钟
result.to_csv('../output/MultinomialNB.csv', index = True, index_label = 'Id' )
# 2.65261